Gradio for met data

In [ ]:
import pandas as pd
import requests
import urllib.parse
import gradio as gr
from io import StringIO
def get_elevation(lat, long):
    """
      Fetch the elevation (in meters) for a given latitude and longitude using the Open-Meteo API.

      Args:
          lat (float): Latitude of the location for which elevation is required.
          long (float): Longitude of the location for which elevation is required.

      Returns:
          float: Elevation in meters for the given latitude and longitude based on the Copernicus DEM 2021 release GLO-90 with 90 meters resolution.
    """
    # Base URL for the Open-Meteo historical weather API
    ELEVATION_API_BASE_URL = "https://api.open-meteo.com/v1/elevation?"

    # Construct the URL parameters for the API request
    ELEVATION_API_URL_PARAMS = {
        "latitude": lat,
        "longitude": long
    }

    # Create the full URL for the API request
    url = ELEVATION_API_BASE_URL + urllib.parse.urlencode(ELEVATION_API_URL_PARAMS, doseq=True)

    # Make the API request
    response = requests.get(url, timeout=5)

    # Get the real elevation
    ELEVATION = response.json()

    return ELEVATION['elevation']


def get_met_data(lat_list, long_list, start_date, end_date):
    """
    Fetch historical weather data from the Open-Meteo API for given locations and date ranges.

    Parameters:
    - lat_list (list of str): List of latitudes for desired locations.
    - long_list (list of str): List of longitudes for desired locations.
    - start_date (list of str): List of start dates in 'YYYY-MM-DD' format.
    - end_date (list of str): List of end dates in 'YYYY-MM-DD' format.

    Returns:
    - DataFrame: Weather data for the given locations and date ranges.

    For available variables, refer to:
    https://open-meteo.com/en/docs/historical-weather-api#models=best_match
    """
    # Base URL for the Open-Meteo historical weather API
    WEATHER_API_BASE_URL = "https://archive-api.open-meteo.com/v1/archive?"

    # Variables of interest for the daily data
    VARIABLES = ["precipitation_sum", "et0_fao_evapotranspiration", "weathercode"]

    # Initialize an empty DataFrame to store the results
    met_data = pd.DataFrame()
    lat_list = [lat_list]
    long_list = [long_list ]
    start_date = [start_date]
    end_date = [end_date]
    # Loop over each location and date range to fetch the data
    for lat, long, start, end in zip(lat_list, long_list, start_date, end_date):
        # Get elevation
        ELEVATION = get_elevation(lat, long)

        # Construct the URL parameters for the API request
        WEATHER_API_URL_PARAMS = {
            "latitude": lat,
            "longitude": long,
            "start_date": start,
            "end_date": end,
            "daily": VARIABLES,
            #   "hourly": "soil_temperature_0_to_7cm",
            "elevation": ELEVATION,
            "timezone": "GMT"
        }

        # Create the full URL for the API request
        url = WEATHER_API_BASE_URL + urllib.parse.urlencode(WEATHER_API_URL_PARAMS, doseq=True)
        # Make the API request
        response = requests.get(url, timeout=5)

        # Parse the JSON response
        data = response.json()

        # Convert the daily data to a DataFrame and add the lat and long columns
        precipitation_sum_3days = sum(
            data["daily"]['precipitation_sum'][-3:])  # Calculate the weather_code on the day before measurement
        weather_code = data["daily"]['weathercode'][-2]  # Calculate the weather_code on the day before measurement
        current_data = pd.DataFrame.from_dict(data["daily"])  # Create a df
        current_data.drop(columns=["time", "weathercode"], inplace=True)
        current_data = current_data.sum(axis=0).to_frame().T  # Calculate the sum for two weeks

        # Add values to DF
        current_data["weathercode_day_before"] = weather_code
        current_data['precipitation_last_3days'] = precipitation_sum_3days
        current_data["lat"], current_data["long"] = lat, long
        current_data["elevation"] = ELEVATION
        current_data["date"] = end
        # sm = pd.DataFrame.from_dict(data["hourly"])

        # Append the current data to the main DataFrame
        met_data = met_data._append(current_data).reset_index(drop=True)

    # Convert the DF toa
    # csv_buffer = StringIO()
    # met_data.to_csv(csv_buffer, index=False)
    # csv_str = csv_buffer.getvalue()
    # Return the final DataFrame containing the weather data
    return met_data,

# Define the Gradio interface
iface = gr.Interface(
    fn=get_met_data,
    inputs=[
        gr.Number(label='Latitude'),
        gr.Number(label='Longitude'),
        gr.Textbox(label='Start Date'),
        gr.Textbox(label='End Date')
    ],
    outputs=gr.Dataframe(label="Generated CSV"),
    live=False  # Because we're generating a file, it's best to set live to False
)
iface.launch()


Elevation Data From Open-Meteo API

In [ ]:
def get_elevation(lat, long):
    """
    Fetch the elevation (in meters) for a given latitude and longitude using the Open-Meteo API.

    Args:
        lat (float): Latitude of the location for which elevation is required.
        long (float): Longitude of the location for which elevation is required.

    Returns:
        float: Elevation in meters for the given latitude and longitude.

    Raises:
        requests.RequestException: If there's an issue with the API request.
        KeyError: If 'elevation' key is not present in the API response.

    Example:
        >>> get_elevation(34.0522, -118.2437)
        71.0  # (This is a hypothetical result, actual output may vary.)

    Note:
        Ensure that the 'requests' and 'urllib.parse' modules are imported before using this function.
    """

    # Base URL for the Open-Meteo historical weather API
    ELEVATION_API_BASE_URL = "https://api.open-meteo.com/v1/elevation?"

    # Construct the URL parameters for the API request
    ELEVATION_API_URL_PARAMS = {
        "latitude": lat,
        "longitude": long
    }

    # Create the full URL for the API request
    url = ELEVATION_API_BASE_URL + urllib.parse.urlencode(ELEVATION_API_URL_PARAMS, doseq=True)

    # Make the API request
    response = requests.get(url)

    # Get the real elevation
    ELEVATION = response.json()
    return ELEVATION['elevation']


Met Data From Open-Meteo API

In [ ]:
import pandas as pd
import requests
import urllib.parse

def get_met_data(lat_list, long_list, start_date, end_date):
    """
    Fetch historical weather data from the Open-Meteo API for given locations and date ranges.

    Parameters:
    - lat_list (list of str): List of latitudes for desired locations.
    - long_list (list of str): List of longitudes for desired locations.
    - start_date (list of str): List of start dates in 'YYYY-MM-DD' format.
    - end_date (list of str): List of end dates in 'YYYY-MM-DD' format.

    Returns:
    - DataFrame: Weather data for the given locations and date ranges.

    For available variables, refer to:
    https://open-meteo.com/en/docs/historical-weather-api#models=best_match
    """

    WEATHER_API_BASE_URL = "https://archive-api.open-meteo.com/v1/archive?"
    VARIABLES = ["precipitation_sum", "et0_fao_evapotranspiration", "weathercode"]
    met_data = pd.DataFrame()

    for lat, long, start, end in zip(lat_list, long_list, start_date, end_date):
        WEATHER_API_URL_PARAMS = {
            "latitude": lat,
            "longitude": long,
            "start_date": start,
            "end_date": end,
            "daily": VARIABLES,
            "timezone": "GMT"
        }

        url = WEATHER_API_BASE_URL + urllib.parse.urlencode(WEATHER_API_URL_PARAMS, doseq=True)
        response = requests.get(url)
        data = response.json()
        current_data = pd.DataFrame.from_dict(data["daily"])
        current_data["lat"] = lat
        current_data["long"] = long
        met_data = pd.concat([met_data, current_data], ignore_index=True)

    return met_data

if __name__ == "__main__":
    lat_list = ["37.71008935", "52.1250655"]
    long_list = ["-122.196217", "4.7563068"]
    start_date = ["2023-04-01", "2023-04-01"]
    end_date = ["2023-04-02", "2023-04-02"]
    data = get_met_data(lat_list, long_list, start_date, end_date)
    print(data)


Get soil Data from soilgrids

In [ ]:
import requests
import urllib.parse
import pandas as pd

def soil_data_df(response, PARAMS):
    """
    Converts API response into a DataFrame for the given soil data parameters.

    Args:
    - response (requests.Response): Response object from the ISRIC API.
    - PARAMS (dict): Dictionary containing query parameters for the API call.

    Returns:
    - DataFrame: A single-row DataFrame containing the soil data.
    """

    # Extract JSON data from the API response
    data = response.json()

    # Determine number of layers and depths based on the parameters
    LAYERS = len(PARAMS["property"])
    DEPTHS = len(PARAMS["depth"])

    row_data = {}  # Initialize an empty dictionary for this lat-long

    # Extract the required soil data values based on the layers and depths
    for layer in range(LAYERS):
        for depth in range(DEPTHS):
            current_depth = PARAMS["depth"][depth]
            current_property = PARAMS["property"][layer]
            value = data["properties"]['layers'][layer]['depths'][depth]['values']['mean']
            layer_name = str(current_property) + " " + str(current_depth)

            row_data[layer_name] = value  # Assign the value to the dictionary

    # Convert the nested dictionary to a DataFrame
    current_lat_long = pd.DataFrame([row_data])

    return current_lat_long

def get_soil_data(lat_list, long_list):
    """
    Fetches soil data from the ISRIC API for the given list of latitudes and longitudes.

    Args:
    - lat_list (list): List of latitudes.
    - long_list (list): List of longitudes.

    Returns:
    - DataFrame: DataFrame containing soil data for all the provided coordinates.
    """

    BASE_URL = "https://rest.isric.org/soilgrids/v2.0/properties/query?"

    # Define the soil properties, depths, and value type to query for
    VARIABLES = ["cfvo", "soc"]
    DEPTH = ["0-5cm", "5-15cm"]
    VALUE = "mean"

    # Initialize an empty DataFrame to store the results
    soil_data = pd.DataFrame()

    # Fetch soil data for each latitude-longitude pair
    for lat, long in zip(lat_list, long_list):
        PARAMS = {
            "lon": long,
            "lat": lat,
            "property": VARIABLES,
            "depth": DEPTH,
            "value": VALUE
        }
        url = BASE_URL + urllib.parse.urlencode(PARAMS, doseq=True)

        response = requests.get(url)

        current_lat_long = soil_data_df(response, PARAMS)
        soil_data = soil_data.append(current_lat_long, ignore_index=True)  # Corrected from '_append' to 'append'

    return soil_data

if __name__ == "__main__":
    pd.set_option('display.max_columns', 7)
    lat_list = ["37.71008935", "52.1250655"]
    long_list = ["-122.196217", "4.7563068"]

    soil_data = get_soil_data(lat_list, long_list)
    print(soil_data)


Returns the date specified days prior to the provided date

In [ ]:
from datetime import datetime, timedelta

def starting_date(date, days=14):
    """
    Returns the date of a specific number of days prior to the provided date.

    This function takes a date in the format 'YYYY-MM-DD', calculates the date
    based on the provided days (default is 14 days) earlier, and returns this earlier date in the same format.

    Parameters:
    - date (str): The input date string in the format 'YYYY-MM-DD'.
    - days (int): Number of days to subtract from the provided date. Default is 14.

    Returns:
    - str: Date in the 'YYYY-MM-DD' format that represents a date 'days' prior
           to the provided input date.

    Examples:
    >>> starting_date("2023-04-15")
    '2023-04-01'
    """
    original_date = datetime.strptime(date, '%Y-%m-%d')
    earlier_date = original_date - timedelta(days=days)
    earlier_date_str = earlier_date.strftime('%Y-%m-%d')
    return earlier_date_str

# Sample data
data = pd.DataFrame({
    'date': ['2023-04-15', '2023-04-20', '2023-05-01']
})

# Get the 3 days before dates
data["days_ago"] = data["date"].apply(starting_date, days=3)
print(data)


binning

In [ ]:
# Get the maximum value from the 'VWC' column
max_value = met_soil_data['VWC'].max()
if max_value < 70:
    max_value = 70

# Defining bin edges and labels
bin_edges = [10, 20, 30, 40, 50, 60, 70, max_value]
bin_labels = ["10", "20", "30", "40", "50", "60", "70", ">70"]

# Binning the data
met_soil_data['VWC_category'] = pd.cut(met_soil_data['VWC'], bins=bin_edges, labels=bin_labels, include_lowest=True)

# Display the original and binned columns
met_soil_data[['VWC', 'VWC_category']].head()


In [ ]:
# Get pixel values`

In [ ]:
from osgeo.gdal import Open
import time
import rasterio

def world_to_pixel_coordinates(geotransform, x, y):
    """
    Convert world coordinates to pixel coordinates.

    Args:
        geotransform (tuple): Geotransformation parameters.
        x (float): X-coordinate in world coordinates.
        y (float): Y-coordinate in world coordinates.

    Returns:
        tuple: Pixel coordinates (px, py).
    """
    px = int((x - geotransform[0]) / geotransform[1])
    py = int((y - geotransform[3]) / geotransform[5])
    return px, py

def get_pixel_value(dataset, px, py, band_number=1):
    """
    Get pixel value from raster dataset at specified pixel coordinates.

    Args:
        dataset (Dataset): Raster dataset.
        px (int): Pixel x-coordinate.
        py (int): Pixel y-coordinate.
        band_number (int, optional): Band number. Defaults to 1.

    Returns:
        int: Pixel value.
    """
    band = dataset.GetRasterBand(band_number)  # Assuming you want the first band
    return band.ReadAsArray(px, py, 1, 1)[0][0]

# After converting to tiff with AGREF
image_file_name = "0000451468_001001_ALOS2411892820-220107/IMG-ALOS2411892820-220107-HBQL1_5RUD_OR_NN_dB_VH.tif"
dataset = Open(image_file_name)
xsize = dataset.RasterXSize
ysize = dataset.RasterYSize
print(f"Image size is {xsize} x {ysize}")
geotransform = dataset.GetGeoTransform()

# Input world coordinates
x, y = -122.196214, 37.71008935

# Convert world coordinates to pixel coordinates using GDAL
px, py = world_to_pixel_coordinates(geotransform, x, y)

# Get pixel value using GDAL
start_time = time.time()
pixel_value = get_pixel_value(dataset, px, py)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"With GDAL, pixel value at world coordinates ({x}, {y}) is: {pixel_value}")
print(f"Time taken: {elapsed_time:.6f} seconds")

# Using Rasterio
with rasterio.open(image_file_name) as src:
    meta = src.meta

    # Use the transform in the metadata and your coordinates
    rowcol = rasterio.transform.rowcol(meta['transform'], xs=x, ys=y)
    start_time = time.time()
    img = src.read(1)
    pixel_value = img[rowcol[0], rowcol[1]]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"With Rasterio, pixel value at world coordinates ({x}, {y}) is: {pixel_value}")
    print(f"Time taken: {elapsed_time:.6f} seconds")


Calculatin MORAN

In [ ]:
import numpy as np
import geopandas as gpd
import pysal.lib
from pysal.explore import esda
from pysal.lib import weights
from shapely.geometry import Point

# Create a GeoDataFrame with some example point geometries
geometry = [Point(xy) for xy in zip(np.random.randint(0, 100, 10), np.random.randint(0, 100, 10))]
geo_df = gpd.GeoDataFrame(geometry=geometry)

# Add an 'ID' column to the GeoDataFrame
geo_df['ID'] = range(1, len(geo_df) + 1)

# Generate some random data values for each point
geo_df['value'] = np.random.rand(len(geo_df))

# Create a spatial weights matrix (here using nearest neighbors)
w = weights.KNN.from_dataframe(geo_df, k=3)

# Compute Local Moran's I
y = geo_df['value'].values
local_moran = esda.Moran_Local(y, w)

# Add Local Moran's I values to the GeoDataFrame
geo_df['Local_Moran_I'] = local_moran.Is

# Print coordinates, ID, and Local Moran's I values
for idx, row in geo_df.iterrows():
    print(f"ID: {row['ID']}, Coordinates: {row['geometry'].coords[:][0]}, Local Moran's I: {row['Local_Moran_I']}")


In [ ]:
import libpysal as lps
import mapclassify as mc
import contextily as ctx
import matplotlib.pyplot as plt

# Filter relevant columns and unique campaigns
subset = gdf[['Campaign','ID','geometry','precipitation_sum_31days_before']]
campaigns = set(gdf['Campaign']) - {'Loch_Eilt_V2', 'Loch_Eilt_V3', 'Netherlands_V2'}

for campaign in campaigns:
    # Filter data for the current campaign
    df = subset[subset["Campaign"] == campaign]

    # Create spatial weights matrix and transform it
    wq = lps.weights.Queen.from_dataframe(df)
    wq.transform = "r"

    # Compute spatial lag and Moran's I
    y = df['precipitation_sum_31days_before']
    ylag = lps.weights.lag_spatial(wq, y)
    moran = esda.Moran(y, wq)

    # Update dataframe with new columns
    df['lag_pre'] = ylag

    # Plotting
    fig, axes = plt.subplots(1, 2, figsize=(2.16 * 4, 4))
    plot_args = {'edgecolor': 'k', 'scheme': 'quantiles', 'k': 3, 'cmap': 'viridis_r', 'legend': True}
    df.plot(column='precipitation_sum_31days_before', ax=axes[0], **plot_args)
    df.plot(column='lag_pre', ax=axes[1], **plot_args)

    for ax, title in zip(axes, [f"Precipitation in campaign {campaign}", "Spatial Lag Precipitation"]):
        ax.axis(df.total_bounds[np.asarray([0, 2, 1, 3])])
        ax.set_title(title)
        ctx.add_basemap(ax, crs='epsg:4326', source=ctx.providers.Stamen.TonerLite, attribution="")

    plt.show()

    # Print Moran's I statistic
    print(f"Moran I: {moran.I:.2f}, p-value: {moran.p_sim:.3f}")


In [ ]:
# This is a placeholder function; actual computation might be more involved
def autocorrelation(data, dist_matrix, max_dist=10000, bin_size=100):
    n_bins = max_dist // bin_size
    acorr = np.zeros(n_bins)

    for i in range(n_bins):
        min_dist, max_dist = i * bin_size, (i+1) * bin_size
        mask = (dist_matrix >= min_dist) & (dist_matrix < max_dist)
        # Compute autocorrelation for this bin
        acorr[i] = np.mean(data[mask])

    return acorr

precip_acorr = autocorrelation(precip_data['value'], dist_matrix)
ET_acorr = autocorrelation(ET_data['value'], dist_matrix)


Closest different neighbour KDTree

In [ ]:
from sklearn.neighbors import KDTree
from geopy.distance import geodesic
import pandas as pd

def calculate_closest_different_neighbors(df, value):
    """
    Find the closest pair of points in a DataFrame with different values for a given column.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the points.
        It must have columns 'lat' and 'long' for latitude and longitude, respectively.

    value : str
        The column name for which we are looking for points with different values.

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the two closest points with different values for the given column.
        If no such pair exists, returns None.

    """
    # Build the KD-Tree
    tree = KDTree(df[['lat', 'long']].values)

    # Initialize variables to find closest points with different values
    min_distance = float('inf')
    closest_points = None

    # Loop through each point
    for i, point in df.iterrows():
        # Query the tree to find closest points
        distances, indices = tree.query([point[['lat', 'long']]], k=len(df))

        # Remove the point itself from the list of closest points
        distances = distances[0][1:]
        indices = indices[0][1:]

        # Check values
        for j, index in enumerate(indices):
            if df.loc[index, value] != point[value]:
                # Convert distances to km using Geopy's geodesic
                point1 = (point['lat'], point['long'])
                point2 = (df.loc[index, 'lat'], df.loc[index, 'long'])
                distance_km = geodesic(point1, point2).km

                if distance_km < min_distance:
                    min_distance = distance_km
                    closest_points = (i, index)
                break

    # Return results
    if closest_points is not None:
        closest_points_df = pd.DataFrame([df.loc[closest_points[0]], df.loc[closest_points[1]]])
        closest_points_df['distance(km)'] = min_distance
        return closest_points_df
    else:
        return None


Fully conected NN

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop_patience = 150
reducelr_patience = 100
reducelr_factor = 0.1
max_epochs = 1000
batch_size = 32
seed = 42

tf.random.set_seed(seed)

early_stopping = EarlyStopping(patience=earlystop_patience, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(patience=reducelr_patience, factor=reducelr_factor)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_met_sat.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.fit(X_train_met_sat, y_train_met_sat,
          epochs=max_epochs,
          batch_size=batch_size,
          validation_split=0.1,
          callbacks=[early_stopping, reduce_lr])

loss, rmse = model.evaluate(X_test_met_sat, y_test_met_sat)


Stacking reggressor

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Generate a synthetic dataset
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
base_models = [
    ('random_forest', RandomForestRegressor(random_state=42)),
    ('xgboost', XGBRegressor(random_state=42))
]

# Define meta-model
meta_model = SVR(kernel='linear', C=1.0)

# Create and fit the stacking regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5)
stacking_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = stacking_regressor.predict(X_test)

# Print the R-squared score of the stacking regressor
print("R-squared score of the stacking regressor: ", stacking_regressor.score(X_test, y_test))


Grid Search for XGboost regressor

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'n_estimators': [50, 100],
    'reg_alpha': [0, 1.0],
    'reg_lambda': [1, 2.0],
    'gamma': [0, 0.5],
}
from sklearn.model_selection import GridSearchCV
xg_reg = xgb.XGBRegressor()

# Assuming xg_reg is your initialized XGBoost regressor
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid,
                          scoring='neg_mean_squared_error', cv=3, verbose=2,
                          n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_search.best_score_)))

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %f" % (mse))
print("RMSE: %f" % (np.sqrt(mse)))



Normalize data sets for LSTM

In [ ]:
def normalize_data(
    train_data: pd.core.frame.DataFrame,
    val_data: pd.core.frame.DataFrame,
    test_data: pd.core.frame.DataFrame,
) -> Tuple[
    np.ndarray, np.ndarray, np.ndarray, pd.core.series.Series, pd.core.series.Series
]:
    """Normalizes train, val and test splits.

    Args:
        train_data (pd.core.frame.DataFrame): Train split.
        val_data (pd.core.frame.DataFrame): Validation split.
        test_data (pd.core.frame.DataFrame): Test split.

    Returns:
        tuple: Normalized splits with training mean and standard deviation.
    """
    train_mean = train_data.mean()
    train_std = train_data.std()

    train_data = (train_data - train_mean) / train_std
    val_data = (val_data - train_mean) / train_std
    test_data = (test_data - train_mean) / train_std

    return train_data, val_data, test_data, train_mean, train_std


plot time series

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

def plot_time_series(feature):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12))
        ax1.plot(train_df["Patv"], color="blue", label="training")
        ax1.plot(val_df["Patv"], color="green", label="validation")
        ax1.plot(test_df["Patv"], color="red", label="test")
        ax1.set_title("Time series of Patv (target)", fontsize=FONT_SIZE_TITLE)
        ax1.set_ylabel("Active Power (kW)", fontsize=FONT_SIZE_AXES)
        ax1.set_xlabel("Date", fontsize=FONT_SIZE_AXES)
        ax1.legend(fontsize=15)
        ax1.tick_params(axis="both", labelsize=FONT_SIZE_TICKS)

        ax2.plot(train_df[feature], color="blue", label="training")
        ax2.plot(val_df[feature], color="green", label="validation")
        ax2.plot(test_df[feature], color="red", label="test")
        ax2.set_title(f"Time series of {feature} (predictor)", fontsize=FONT_SIZE_TITLE)
        ax2.set_ylabel(f"{feature}", fontsize=FONT_SIZE_AXES)
        ax2.set_xlabel("Date", fontsize=FONT_SIZE_AXES)
        ax2.legend(fontsize=15)
        ax2.tick_params(axis="both", labelsize=FONT_SIZE_TICKS)

        plt.tight_layout()
        plt.show()

 feature_selection = widgets.Dropdown(
        options=[f for f in list(train_df.columns) if f != "Patv"],
        description="Feature",
    )

interact(plot_time_series, feature=feature_selection)

compute mmetrics between time series

In [ ]:
def compute_metrics(
    true_series: np.ndarray, forecast: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """Computes MSE and MAE between two time series.

    Args:
        true_series (np.ndarray): True values.
        forecast (np.ndarray): Forecasts.

    Returns:
        tuple: MSE and MAE metrics.
    """

    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

Creates a Conv-LSTM model

In [ ]:

def create_model(num_features: int, days_in_past: int) -> tf.keras.Model:
    """Creates a Conv-LSTM model for time series prediction.

    Args:
        num_features (int): Number of features used for prediction.
        days_in_past (int): Number of days into the past to predict next 24 hours.

    Returns:
        tf.keras.Model: The uncompiled model.
    """
    CONV_WIDTH = 3
    OUT_STEPS = 24
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Masking(
                mask_value=-1.0, input_shape=(days_in_past * 24, num_features)
            ),
            tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
            tf.keras.layers.Conv1D(256, activation="relu", kernel_size=(CONV_WIDTH)),
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(32, return_sequences=True)
            ),
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(32, return_sequences=False)
            ),
            tf.keras.layers.Dense(
                OUT_STEPS * 1, kernel_initializer=tf.initializers.zeros()
            ),
            tf.keras.layers.Reshape([OUT_STEPS, 1]),
        ]
    )

    return model


Compile and fit model

In [ ]:
def compile_and_fit(
    model: tf.keras.Model, window: WindowGenerator, patience: int = 2
) -> tf.keras.callbacks.History:
    """Compiles and trains a model given a patience threshold.

    Args:
        model (tf.keras.Model): The model to train.
        window (WindowGenerator): The windowed data.
        patience (int, optional): Patience threshold to stop training. Defaults to 2.

    Returns:
        tf.keras.callbacks.History: The training history.
    """
    EPOCHS = 20

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=patience, mode="min"
    )

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(),
    )

    tf.random.set_seed(432)
    np.random.seed(432)
    random.seed(432)

    history = model.fit(
        window.train, epochs=EPOCHS, validation_data=window.val, callbacks=[early_stopping]
    )

    if len(history.epoch) < EPOCHS:
        print("\nTraining stopped early to prevent overfitting, as the validation loss is increasing for two consecutive steps.")

    return history


Train model

In [ ]:

def train_conv_lstm_model(
    data: pd.core.frame.DataFrame, features: List[str], days_in_past: int
) -> Tuple[WindowGenerator, tf.keras.Model, DataSplits]:
    """Trains the Conv-LSTM model for time series prediction.

    Args:
        data (pd.core.frame.DataFrame): The dataframe to be used.
        data (list[str]): The features to use for forecasting.
        days_in_past (int): How many days in the past to use to forecast the next 24 hours.

    Returns:
        tuple: The windowed dataset, the model that handles the forecasting logic and the data used.
    """
    data_splits = train_val_test_split(data[features])

    train_data, val_data, test_data, train_mean, train_std = (
        data_splits.train_data,
        data_splits.val_data,
        data_splits.test_data,
        data_splits.train_mean,
        data_splits.train_std,
    )

    window = generate_window(train_data, val_data, test_data, days_in_past)
    num_features = window.train_df.shape[1]

    model = create_model(num_features, days_in_past)
    history = compile_and_fit(model, window)

    return window, model, data_splits


Create forecast

In [ ]:

def add_wind_speed_forecasts(
    df: pd.core.frame.DataFrame, add_noise=False
) -> pd.core.frame.DataFrame:
    """Creates syntethic wind speed forecasts. The more into the future, the more noise these have.

    Args:
        df (pd.core.frame.DataFrame): Dataframe with data from turbine.
        periods (list, optional): Periods for which to create the forecast. Defaults to [*range(1, 30, 1)].

    Returns:
        pd.core.frame.DataFrame: The new dataframe with the synth forecasts.
    """

    df_2 = df.copy(deep=True)
    # Periods for which to create the forecast.
    periods=[*range(1, 30, 1)]

    for period in periods:

        if add_noise == "linearly_increasing":
            np.random.seed(8752)
            noise_level = 0.2 * period
            noise = np.random.randn(len(df)) * noise_level

        elif add_noise == "mimic_real_forecast":
            np.random.seed(8752)
            noise_level = 2 + 0.05 * period
            noise = np.random.randn(len(df)) * noise_level
        else:
            noise = 0

        padding_slice = df_2["Wspd"][-period:].to_numpy()
        values = np.concatenate((df_2["Wspd"][period:].values, padding_slice)) + noise

        df_2[f"fc-{period}h"] = values

    return df_2

In [ ]:

def plot_forecast_with_noise(
    data_with_wspd_forecasts: pd.core.frame.DataFrame,
) -> None:
    """Creates an interactive plot that shows how the synthetic forecasts change when the future horizon is changed.

    Args:
        data_with_wspd_forecasts (pd.core.frame.DataFrame): Dataframe that includes synth forecasts.
    """

    def _plot(noise_level):
        fig, ax = plt.subplots(figsize=(20, 6))

        df = data_with_wspd_forecasts
        synth_data = df[f"fc-{noise_level}h"][
            5241 - noise_level : -noise_level
        ].values
        synth_data = tf.nn.relu(synth_data).numpy()
        real_data = df["Wspd"][5241:].values
        real_data = tf.nn.relu(real_data).numpy()

        mae = metrics.mean_absolute_error(real_data, synth_data)

        print(f"\nMean Absolute Error (m/s): {mae:.2f} for forecast\n")
        ax.plot(df.index[5241:], real_data, label="true values")
        ax.plot(
            df.index[5241:],
            synth_data,
            label="syntethic predictions",
        )

        ax.set_title("Generated wind speed forecasts", fontsize=FONT_SIZE_TITLE)
        ax.set_ylabel("Wind Speed (m/s)", fontsize=FONT_SIZE_AXES)
        ax.set_xlabel("Date", fontsize=FONT_SIZE_AXES)
        ax.tick_params(axis="both", labelsize=FONT_SIZE_TICKS)
        ax.legend()

    noise_level_selection = widgets.IntSlider(
        value=1,
        min=1,
        max=25,
        step=1,
        description="Noise level in m/s (low to high)",
        disabled=False,
        continuous_update=False,
        orientation="horizontal",
        readout=False,
        layout={"width": "500px"},
        style={"description_width": "initial"},
    )

    interact(_plot, noise_level=noise_level_selection)

In [ ]:

def window_plot(data_splits: DataSplits) -> None:
    """Creates an interactive plots to show how the data is windowed depending on the number of days into the past that are used to forecast the next 24 hours.

    Args:
        data_splits (DataSplits): Data used.
    """
    train_data, val_data, test_data = (
        data_splits.train_data,
        data_splits.val_data,
        data_splits.test_data,
    )

    def _plot(time_steps_past):
        window = generate_window(train_data, val_data, test_data, time_steps_past)
        window.plot()

    time_steps_past_selection = widgets.IntSlider(
        value=1,
        min=1,
        max=14,
        step=1,
        description="Days before",
        disabled=False,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format="d",
        layout={"width": "500px"},
        style={"description_width": "initial"},
    )

    interact(_plot, time_steps_past=time_steps_past_selection)



Plot forecast

In [ ]:
def plot_forecast(weather_forecasts: Dict[str, Dict[List[datetime], List[float]]]) -> None:
    """Creates an interactive plot of true values vs forecasts for the wind data.

    Args:
        weather_forecasts (dict): History of weather and weather forecasts.
    """
    def _plot(city, time_steps_future):
        format_timestamp = "%Y-%m-%d %H:%M:%S"

        weather_forecast = weather_forecasts[city]

        dates_real, winds_real = weather_forecast[0]
        dates_real = [datetime.strptime(i, format_timestamp) for i in dates_real]
        dates_forecast, winds_forecast = weather_forecast[time_steps_future]
        dates_forecast = [datetime.strptime(i, format_timestamp) for i in dates_forecast]

        # Set the min and max date for plotting, so it always plots the same
        min_date = datetime.strptime("2022-11-16 18:00:00", format_timestamp)
        max_date = datetime.strptime("2023-01-11 15:00:00", format_timestamp)

        # Find the overlap of the data and limit it to the plotting range
        dates_real, dates_forecast, winds_real, winds_forecast = prepare_wind_data(
            dates_real, dates_forecast, winds_real, winds_forecast, min_date, max_date
        )

        fig, ax = plt.subplots(figsize=(20, 6))
        ax.plot(dates_real, winds_real, label="Actual windspeed")
        ax.plot(dates_forecast, winds_forecast, label=f"Forecasted windspeed {time_steps_future} Hours in the Future")
        ax.set_title(f"History of Actual vs Forecasted Windspeed in {city}", fontsize=25)
        ax.set_ylabel("Wind Speed (m/s)", fontsize=20)
        ax.set_xlabel("Date", fontsize=20)
        ax.tick_params(axis="both", labelsize=15)
        ax.legend(fontsize=15)

        mae = metrics.mean_absolute_error(winds_real, winds_forecast)
        print(f"\nMean Absolute Error (m/s): {mae:.2f} for forecast\n")

    city_selection = widgets.Dropdown(
        options=weather_forecasts.keys(),
        description='City',
    )
    time_steps_future_selection = widgets.IntSlider(
        value=1,
        min=3,
        max=120,
        step=3,
        description="Hours into future",
        disabled=False,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format="d",
        layout={"width": "500px"},
        style={"description_width": "initial"},
    )

    interact(_plot, city=city_selection, time_steps_future=time_steps_future_selection)

add project to clear ml

In [ ]:
# in terminal first clearml-init
from clearml import Task
task = Task.init(project_name = "Wind Speed Predictor", task_name = 'tesorflow_training')